In [1]:
from numpy import loadtxt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#load data
tr_data = pd.read_table("shuf_total_train_human_mouse", sep='\t', encoding='utf-8', engine='python')
tr_data = tr_data.drop(['#ID'], axis=1)

In [4]:
ts_hum_data = pd.read_table("shuf_tot_human", sep='\t', encoding='utf-8', engine='python')
ts_hum_data = ts_hum_data.drop(['#ID'], axis=1)
ts_zebrafish_data = pd.read_table("shuf_tot_zebrafish", sep='\t', encoding='utf-8', engine='python')
ts_zebrafish_data = ts_zebrafish_data.drop(['#ID'], axis=1)
ts_mouse_data = pd.read_table("shuf_tot_mouse", sep='\t', encoding='utf-8', engine='python')
ts_mouse_data = ts_mouse_data.drop(['#ID'], axis=1)
ts_gen_data = pd.read_table("fin_shuf_gen", sep='\t', encoding='utf-8', engine='python')
ts_gen_data = ts_gen_data.drop(['#ID'], axis=1)
ts_mouse_gen_data = pd.read_table("total_v18_mouse_feat", sep='\t', encoding='utf-8', engine='python')
ts_mouse_gen_data = ts_mouse_gen_data.drop(['#ID'], axis=1)

In [5]:
tr_model = pd.get_dummies(tr_data)

In [6]:
ts_hum_model = pd.get_dummies(ts_hum_data)
ts_mouse_model = pd.get_dummies(ts_mouse_data)
ts_zebrafish_model = pd.get_dummies(ts_zebrafish_data)
ts_gen_model = pd.get_dummies(ts_gen_data)
ts_mouse_gen_model = pd.get_dummies(ts_mouse_gen_data)


In [7]:
prot_feat = ['instability', 'gravy', 'Mw', 'pI', 'PW', 'peptide_length']
rna_feat = ['ORF_coverage', 'ORF_integrity', 'GC1', 'GC2', 'GC3']
dna_feat = ['transcript_length', 'stop_codon_num', 'Fickett_score']

label = ['#label_noncoding']



y_train = tr_model[label]

x_prot_train = tr_data[prot_feat]
x_rna_train = tr_data[rna_feat]
x_dna_train = tr_data[dna_feat]

final_feats = []

f_test = ['transcript_length', 'Fickett_score', 'ORF_coverage', 'ORF_integrity', 'GC1', 'GC2', 'GC3', 'stop_codon_num', 'instability', 'gravy', 'pI', 'Mw', 'PW']
l_test = ['#label_noncoding']

In [8]:
#model = XGBClassifier(silent=True,  scale_pos_weight=1,learning_rate=0.2,colsample_bytree = 0.4,subsample = 0.8,objective='binary:logistic', n_estimators=1000, 
#reg_alpha = 0.3,max_depth=10, gamma=5)
model = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=7, min_child_weight=0, gamma=0, subsample=0.9, 
                      colsample_bytree=0.6, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
#model = XGBClassifier(gamma=4, max_depth = 10, learning_rate=0.2)

In [9]:
fin_fin_feats = ['gravy', 'Mw', 'pI', 'GC3', 'Fickett_score']

x_train = tr_model[fin_fin_feats]

x_hum_test = ts_hum_model[fin_fin_feats]
y_hum_test = ts_hum_model[l_test]

x_mouse_test = ts_mouse_model[fin_fin_feats]
y_mouse_test = ts_mouse_model[l_test]

x_zebrafish_test = ts_zebrafish_model[fin_fin_feats]
y_zebrafish_test = ts_zebrafish_model[l_test]

x_gen_test = ts_gen_model[fin_fin_feats]
y_gen_test = ts_gen_model[l_test]

x_mouse_gen_test = ts_mouse_gen_model[fin_fin_feats]
y_mouse_gen_test = ts_mouse_gen_model[l_test]

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
kf.get_n_splits(x_train)
#KFold(n_splits=2, random_state=None, shuffle=False)
cnt = 0
for train_index, test_index in kf.split(x_train):
    cnt += 1
    #print("TRAIN:", train_index, "TEST:", test_index)
    xx_train, xx_test = x_train.iloc[train_index], x_train.iloc[test_index]
    yy_train, yy_test = y_train.iloc[train_index], y_train.iloc[test_index]
    model.fit(xx_train, yy_train)
    yy_pred = model.predict(xx_test)
    accuracy = accuracy_score(yy_test, yy_pred)
    print("Fold : %d"%cnt)
    print("Accuracy: %.2f%%"%(accuracy * 100.0))

/home/thesis/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/thesis/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold : 1
Accuracy: 90.23%
Fold : 2
Accuracy: 90.52%
Fold : 3
Accuracy: 91.20%
Fold : 4
Accuracy: 89.80%
Fold : 5
Accuracy: 90.40%
Fold : 6
Accuracy: 90.20%
Fold : 7
Accuracy: 90.53%
Fold : 8
Accuracy: 90.40%
Fold : 9
Accuracy: 90.43%
Fold : 10
Accuracy: 90.03%


In [11]:
#normal
y_hum_pred = model.predict(x_hum_test)
y_mouse_pred = model.predict(x_mouse_test)
y_zebrafish_pred = model.predict(x_zebrafish_test)
y_gen_pred = model.predict(x_gen_test)
y_mouse_gen_pred = model.predict(x_mouse_gen_test)

In [12]:
accuracy = accuracy_score(y_hum_test, y_hum_pred)
print("Accuracy human: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(y_mouse_test, y_mouse_pred)
print("Accuracy mouse: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(y_zebrafish_test, y_zebrafish_pred)
print("Accuracy zebrafish: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(y_gen_test, y_gen_pred)
print("Accuracy gen: %.2f%%" % (accuracy * 100.0))

accuracy = accuracy_score(y_mouse_gen_test, y_mouse_gen_pred)
print("Accuracy mouse gen: %.2f%%" % (accuracy * 100.0))

Accuracy human: 95.80%
Accuracy mouse: 93.81%
Accuracy zebrafish: 86.22%
Accuracy gen: 92.51%
Accuracy mouse gen: 96.09%


In [ ]:
Accuracy human: 95.80%
Accuracy mouse: 93.81%
Accuracy zebrafish: 86.22%
Accuracy gen: 92.51%
Accuracy mouse gen: 96.09%

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class non coding', 'class coding']
print(classification_report(y_hum_test, y_hum_pred, target_names=target_names))

#target_names = ['class non coding', 'class coding']
print(classification_report(y_mouse_test, y_mouse_pred, target_names=target_names))

#target_names = ['class non coding', 'class coding']
print(classification_report(y_zebrafish_test, y_zebrafish_pred, target_names=target_names))

print(classification_report(y_gen_test, y_gen_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
from __future__ import division
tn, fp, fn, tp = confusion_matrix(y_hum_test, y_hum_pred).ravel()
specificity = tn / (tn+fp)
print(specificity)

tn, fp, fn, tp = confusion_matrix(y_mouse_test, y_mouse_pred).ravel()
specificity = tn / (tn+fp)
print(specificity)

tn, fp, fn, tp = confusion_matrix(y_zebrafish_test, y_zebrafish_pred).ravel()
specificity = tn / (tn+fp)
print(specificity)

tn, fp, fn, tp = confusion_matrix(y_gen_test, y_gen_pred).ravel()
specificity = tn / (tn+fp)
print(specificity)

In [ ]:
from sklearn.metrics import confusion_matrix

cm_hum = confusion_matrix(y_hum_test, y_hum_pred)
print(cm_hum)
cm_mouse = confusion_matrix(y_mouse_test, y_mouse_pred)
print(cm_mouse)
cm_zebrafish = confusion_matrix(y_zebrafish_test, y_zebrafish_pred)
print(cm_zebrafish)
cm_gen = confusion_matrix(y_gen_test, y_gen_pred)
print(cm_gen)

In [ ]:
from sklearn.metrics import roc_auc_score

print roc_auc_score(y_hum_test, y_hum_pred)

print roc_auc_score(y_mouse_test, y_mouse_pred)

print roc_auc_score(y_zebrafish_test, y_zebrafish_pred)

print roc_auc_score(y_gen_test, y_gen_pred)

In [ ]:
#precision : the ability of the classifier not to label as positive a sample that is negative.
#recall : the ability of the classifier to find all the positive samples
#f1-score :  F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0
# support : The support is the number of occurrences of each class in y_true.

In [ ]:
from sklearn.metrics import roc_curve
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_hum_test, y_hum_pred)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0,1],[0,1], 'r--')
plt.yticks(np.arange(0.0, 1.2, step=0.05))
plt.xticks(np.arange(0.0, 1.2, step=0.1))
#print(lock, labels)
plt.xlim([0.0,1.1])
plt.ylim([0.0,1.1])
plt.ylabel('True Positive Rate(Sensitivity)')
plt.xlabel('False Positive Rate(Specificity)')
plt.autoscale(enable = False, axis = 'both', tight = True)
plt.rcParams['figure.figsize'] = [6, 5]
#print(plt.grid)
#plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_mouse_test, y_mouse_pred)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0,1],[0,1], 'r--')
plt.yticks(np.arange(0.0, 1.2, step=0.05))
plt.xticks(np.arange(0.0, 1.2, step=0.1))
#print(lock, labels)
plt.xlim([0.0,1.1])
plt.ylim([0.0,1.1])
plt.ylabel('True Positive Rate(Sensitivity)')
plt.xlabel('False Positive Rate(Specificity)')
plt.autoscale(enable = False, axis = 'both', tight = True)
plt.rcParams['figure.figsize'] = [6, 5]
#print(plt.grid)
#plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_zebrafish_test, y_zebrafish_pred)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0,1],[0,1], 'r--')
plt.yticks(np.arange(0.0, 1.2, step=0.05))
plt.xticks(np.arange(0.0, 1.2, step=0.1))
#print(lock, labels)
plt.xlim([0.0,1.1])
plt.ylim([0.0,1.1])
plt.ylabel('True Positive Rate(Sensitivity)')
plt.xlabel('False Positive Rate(Specificity)')
plt.autoscale(enable = False, axis = 'both', tight = True)
plt.rcParams['figure.figsize'] = [6, 5]
#print(plt.grid)
#plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_gen_test, y_gen_pred)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0,1],[0,1], 'r--')
plt.yticks(np.arange(0.0, 1.2, step=0.05))
plt.xticks(np.arange(0.0, 1.2, step=0.1))
#print(lock, labels)
plt.xlim([0.0,1.1])
plt.ylim([0.0,1.1])
plt.ylabel('True Positive Rate(Sensitivity)')
plt.xlabel('False Positive Rate(Specificity)')
plt.autoscale(enable = False, axis = 'both', tight = True)
plt.rcParams['figure.figsize'] = [6, 5]
#print(plt.grid)
#plt.figure(figsize=(20, 10))
plt.show()